In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaTokenizer, LlamaForCausalLM

base_model_path = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    #quantization_config=bnb_config,
    #device_map="auto",
    use_cache=False
)

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [2]:
tokenizer = AutoTokenizer.from_pretrained(base_model_path)

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [5]:
text = "Sample input."

input_ids = tokenizer(text, return_tensors="pt").input_ids
%time output = model(input_ids)
print(output.logits)

CPU times: user 1.9 s, sys: 0 ns, total: 1.9 s
Wall time: 124 ms
tensor([[[ -4.6822,   0.9866,   4.5126,  ...,  -5.2010,  -2.1646,  -4.2286],
         [-10.1421, -10.1378,   5.6231,  ...,  -6.1040, -11.0720,  -4.8641],
         [ -9.6640,  -9.2623,   4.9745,  ...,  -7.9037, -11.2208,  -3.2938],
         [ -8.7681,  -8.3673,   8.2792,  ...,  -5.3513,  -8.6309,  -3.2394]]],
       grad_fn=<UnsafeViewBackward0>)


In [1]:
from transformers.models.zamba2 import Zamba2Config, Zamba2Model
config = Zamba2Config(# hidden_size=256,
                    use_cache=False,
                    use_mamba_kernels=True)

model = Zamba2Model(config)
model = model.cuda()


In [2]:
import torch
inp = torch.tensor([[1, 2]]).cuda()
model(inp)

Zamba requires an initialized `HybridMambaAttentionDynamicCache` to return a cache. None was provided, so no cache will be returned.


<function MLP.__init__.<locals>.glu at 0x720f6c155120>
tensor([[[ 0.2320, -0.0671,  0.4281,  ...,  0.2731, -0.2485, -0.1961],
         [-0.3529, -0.3817,  0.1831,  ...,  0.4370,  0.1053,  0.2115]]],
       device='cuda:0', grad_fn=<AddBackward0>)
<function MLP.__init__.<locals>.glu at 0x720e33f02b90>
tensor([[[-0.2456, -0.1861, -0.3472,  ...,  0.2924, -0.3361,  0.0805],
         [-0.2854, -0.3549,  0.2099,  ...,  0.2326, -0.2619,  0.0898]]],
       device='cuda:0', grad_fn=<AddBackward0>)
<function MLP.__init__.<locals>.glu at 0x720f6c155120>
tensor([[[ 0.3210,  0.0659,  0.5195,  ...,  0.2646, -0.4936, -0.3867],
         [-0.0319,  0.0588,  0.0368,  ...,  0.2150, -0.6552, -0.3597]]],
       device='cuda:0', grad_fn=<AddBackward0>)
<function MLP.__init__.<locals>.glu at 0x720e33f02b90>
tensor([[[-0.2532, -0.1881, -0.3141,  ...,  0.6968, -0.3180, -0.1367],
         [-0.5472, -0.2829,  0.1529,  ...,  0.4406, -0.1009, -0.1136]]],
       device='cuda:0', grad_fn=<AddBackward0>)
<function ML

BaseModelOutputWithPast(last_hidden_state=tensor([[[ 1.5610, -0.2107, -0.1899, -0.6792,  0.1004, -1.4207,  0.4914,
          -2.6973, -1.5301,  0.8072,  0.1398,  0.3223,  0.7193, -0.6386,
          -0.0224, -0.0305,  1.0239,  1.1427,  0.5567, -1.2266,  1.0427,
          -0.2688,  0.3386, -0.3193, -0.7158,  0.4627, -0.0887, -0.3950,
           0.4347,  0.6391, -0.1739, -1.6352,  1.1399,  0.4062,  0.2984,
          -1.9093,  0.3013,  2.0630, -0.8852, -0.8784,  0.8980, -0.7977,
           0.8236, -0.5900, -0.1444,  1.0169, -0.0153,  2.7408, -1.3365,
          -0.1727, -0.4123, -2.5188,  0.4768, -1.0458,  0.2904, -0.3563,
           1.6769, -0.3954, -0.3084, -0.5172, -1.7214,  1.7523,  0.7369,
           1.1858,  0.1346,  0.0534,  0.9420,  1.2791, -0.2385,  0.3122,
           0.8081, -1.1902,  1.1936,  0.7331, -1.4534,  0.5935,  0.7391,
          -0.9888, -0.2075,  0.6612,  0.8746, -0.3961, -0.5769, -1.5450,
           1.4662, -0.3326,  1.2070, -0.0269, -0.3110,  1.7675,  0.9386,
         